In [1]:
from torchvision.models import *
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Multiclass-image-dataset-airplane-car-ship'

In [61]:
def load_data():
    labels = {}
    labels_r = {}
    idx = -1
    data = []
    for folder in os.listdir('./data/'):
        idx += 1
        labels[folder] = idx
        labels_r[idx] = folder
    print(labels)
    for folder in tqdm(os.listdir('./data/')):
        print(folder)
        print(np.eye(labels[folder]+1,idx+1)[labels[folder]])
        for file in os.listdir(f'./data/{folder}/'):
            img = cv2.imread(f'./data/{folder}/{file}')
            img = cv2.resize(img,(56,56))
            img = img / 255.0
            data.append([img,np.eye(labels[folder]+1,idx+1)[labels[folder]]])
    np.random.shuffle(data)
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)
    return X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data

In [62]:
X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data = load_data()
X_train = torch.from_numpy(np.array(X_train)).view(-1,3,56,56).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).view(-1,3,56,56).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

{'cars': 0, 'ship': 1, 'airplanes': 2}


  0%|                                                     | 0/3 [00:00<?, ?it/s]

cars
[1. 0. 0.]


 33%|███████████████                              | 1/3 [00:12<00:24, 12.50s/it]

ship
[0. 1. 0.]


 67%|██████████████████████████████               | 2/3 [00:12<00:05,  5.36s/it]

airplanes
[0. 0. 1.]


100%|█████████████████████████████████████████████| 3/3 [00:22<00:00,  7.66s/it]


In [63]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [64]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        y_batch = int(torch.argmax(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [76]:
model = resnet18().to(device)

In [77]:
model.fc = Linear(512,3)

In [78]:
model.fc

Linear(in_features=512, out_features=3, bias=True)

In [79]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [80]:
criterion = MSELoss()

In [81]:
batch_size = 32
epochs = 1000

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size].to(device).float()
        y_batch = y_train[i:i+batch_size].to(device).float()
        model.to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion))/2})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    wandb.log({'Val ACC':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()

 89%|███████████████████████████████████▊    | 894/1000 [24:37<02:57,  1.67s/it]